# The Online Cafeteria
## Final Exam in Programming, Algorithms and Data Structures
### Daniel Henke, 176182

#### Customer Class

In [7]:
from abc import ABC, abstractmethod
#using abstract class to define general customer class that will be inherited by other classes

class Customer(ABC):
    @abstractmethod
    def init(self, name, customer_id, customer_type, discount=0):
        self.name = name
        self.customer_id = customer_id
        self.customer_type = customer_type
        self.discount = discount
        self.balance = 0
        self.orders = []
    
    def str(self):
        if self.customer_id == None:
            return f"{self.name} ({self.customer_type})"   
        else:
            return f"{self.name} - {self.customer_id} ({self.customer_type})"    
    
    def view_menu(self, cafeteria):
        """Lets the customer view the menu of a cafeteria and the prices of the items after applying the discount.

        Args:
            cafeteria (object): the cafeteria object whose menu the customer wants to view
        """
        print(f"Menu for {cafeteria.name}:")
        for item, details in cafeteria.menu.items():
            print(f"{item}: {details['price']*(1-(self.discount//100))}dkk ({details['quantity']} available)")
    
    def view_detailed_menu(self, cafeteria):
        """Lets the customer view the detailed menu of a cafeteria.

        Args:
            cafeteria (object): the cafeteria object whose menu the customer wants to view
        """
        print(f"Menu for {cafeteria.name}:")
        for item, details in cafeteria.menu.items():
            print(f"{item}: {details['description']}")
            print(f"{details['price']}dkk ({details['quantity']} available)")
    
    def add_balance(self, amount):
        """Adds balance to the account.

        Args:
            amount (int): the amount to be added to the balance
        """
        self.balance+=amount
        print(f"${amount} added to the balance of {self.name}")
         
    def place_order(self, cafeteria, item, quantity):
        """Places an order in a cafeteria.

        Args:
            cafeteria (object): the cafeteria object where the order is to be placed
            item (string): the item to be ordered
            quantity (int): number of items to be ordered
        """
        if item in cafeteria.menu:
            #calculating the price after discount for full order           
            price = cafeteria.menu[item]['price']*quantity*(1-(self.discount//100))
            if self.balance < price:
                raise ValueError(f"Sorry, you do not have enough balance to place this order")
            
            #placing the order with the cafeteria
            order=cafeteria.process_order(self.customer_id, item, quantity, self.discount)
            
            #updating the balance based on the actual order (may be lower quantity)
            self.balance-=order.price
            self.orders.append(order)
            print(f"Order placed by {self.name} for {order.quantity} {item}(s) for ${order.price}")
                
        else:
            raise ValueError(f"Sorry, {item} is not available in the menu")
        

#### Student
The first subclass of customer

In [8]:
class Student(Customer):
    def __init__(self, name, student_id):
        super().__init__(name, student_id, "Student", 20)

#### Staff

In [ ]:
class Staff(Customer):
    def __init__(self, name, staff_id):
        super().__init__(name, staff_id, "Staff", 10)

#### Guest
Guests cannot order online and have no account. They can merely view the menu.

In [ ]:
class Guest(Customer):
    def __init__(self, name):
        super().__init__(name, None, "Guest", 0)
    
    def place_order(self, cafeteria, item, quantity):
        raise PermissionError("Guests cannot place orders online. Please book in person.")
    
    def add_balance(self, amount):
        raise PermissionError("Guests do not have a balance to add to.")
    

#### Cafeteria

In [ ]:
class Cafeteria:
    def __init__(self, name, university):
        self.name = name
        self.university = university
        self.menu = {}
        self.orders = []
        self.item_popularity = {}
        self.revenue=0
        
    def add_item(self, item, description, price, quantity):
        """Adds an item to the menu of the cafeteria.

        Args:
            item (string): the name of the item
            description (string): the description of the item
            price (int): the price of the item
            quantity (int): the quantity of the item available
        """
        self.menu[item] = {'description': description, 'price': price, 'quantity': quantity}
        self.university.update_sorted_menu(item, quantity, self.name, description, price)
        
    
    def upload_menu(self, menu):
        """Uploads the menu to the cafeteria.

        Args:
            menu (dict): a dictionary containing the items and their details
        """
        self.menu = menu
        self.university.is_sorted = False
        
    def update_item(self, item, description, price, quantity):
        """Updates the price and quantity of an item in the menu
        Args:
            item (string): the name of the item
            description (string): the new description of the item
            price (int): the new price of the item
            quantity (int): the new quantity of the item
        """
        if item in self.menu:
            self.menu[item]['description'] = description
            self.menu[item]['price'] = price
            self.menu[item]['quantity'] = quantity
            self.university.is_sorted = False
        else:
            raise ValueError(f"Sorry, {item} is not available in the menu")
        
    def restock_item(self, item, quantity):
        """Restocks an item in the menu.

        Args:
            item (string): the name of the item
            quantity (int): the quantity to be added to the stock
        """
        if item in self.menu:
            self.menu[item]['quantity']+=quantity
            self.university.update_sorted_menu(item, self.menu[item]['quantity'], self.name, self.menu[item]['description'], self.menu[item]['price'])
        else:
            raise ValueError(f"Sorry, {item} is not available in the menu")
        
    def remove_item(self, item):
        """Removes an item from the menu.

        Args:
            item (string): the name of the item
        """
        if item in self.menu:
            self.menu.pop(item)
        else:
            raise ValueError(f"Sorry, {item} is not available in the menu")
    
    def process_order(self, customer_id, item, quantity, discount=0):
        """Processes an order placed by a customer.

        Args:
            customer_id (int): the id of the customer placing the order
            item (string): the item to be ordered
            quantity (int): number of items to be ordered

        Returns:
            object: the order object
        """
        if item in self.menu:
            if self.menu[item]['quantity'] < quantity and self.menu[item]['quantity'] > 0:
                print(f"Sorry, only {self.menu[item]['quantity']} {item}(s) available")
                quantity = self.menu[item]['quantity']
            elif self.menu[item]['quantity'] == 0:
                raise ValueError(f"Sorry, {item} is out of stock")
            
            order = Order(self, customer_id, item, quantity, self.menu[item]['price'], discount)
            if item not in self.item_popularity:
                self.item_popularity[item] = 0
            self.item_popularity[item] += quantity
            self.orders.append(order)
            self.print_orders()
            return order
        else:
            raise ValueError(f"Sorry, {item} is not available in the menu")
        
    def print_orders(self):
        """Prints all the orders placed in the cafeteria."""
        print(f"Orders in {self.name}:")
        for order in self.orders:
            print(order)
            
    
    def complete_order(self, order_id):
        """Completes an order placed by a customer.

        Args:
            order (int): the order object to be completed
        """
        for order in self.orders:
            if order.order_id == order_id:
                self.orders.remove(order)
                self.menu[order.item]['quantity']-=order.quantity
                self.university.update_sorted_menu(order.item, order.quantity, self.name)
                self.revenue+=order.price
                order.complete()
        return f"Order {order_id} not found"
        
    
    def cancel_order(self, order_id):
        """Cancels an order placed by a customer.

        Args:
            order (int): the order object to be cancelled
        """
        for order in self.orders:
            if order.order_id == order_id:
                self.orders.remove(order)
                order.cancel()
        return f"Order {order_id} not found"
    
    def close_cafeteria(self):
        """Closes the cafeteria for the day and prints the revenue generated."""
        print(f"Revenue generated today by {self.name}: ${self.revenue}")
        for order in self.orders:
            order.cancel()
        self.item_popularity = {}
        self.menu = {}
        self.university.is_sorted = False
        self.revenue = 0
    
    def popular_items(self, n):
        """Prints the n most popular items in the cafeteria.

        Args:
            n (int): the number of items to be printed
        """
        
        def merge_sort(items):
            if len(items) <= 1:
                return items

            # Split into two halves
            mid = len(items) // 2
            left_half = merge_sort(items[:mid])
            right_half = merge_sort(items[mid:])

            # Merge sorted halves
            return merge(left_half, right_half)

        def merge(left, right):
            sorted_items = []
            while len(left)>0 and len(right)>0:
                # Compare the second element (popularity)
                if left[0][1] >= right[0][1]:
                    sorted_items.append(left.pop(0))
                else:
                    sorted_items.append(right.pop(0))

            # Append any remaining items
            if len(left) > 0:
                sorted_items.extend(left)
            elif len(right) > 0:
                sorted_items.extend(right)
            return sorted_items
                
        
        # Convert dictionary to a list of tuples (item, popularity)
        items = list(self.item_popularity.items())
        # Sort using merge sort
        sorted_items = merge_sort(items)
        # Return the top N items
        return sorted_items[:n]

#### Order

In [ ]:
class Order:
    class_counter=1
    def __init__(self, cafeteria, customer_id, item, quantity, price, discount=0):
        self.order_id = Order.class_counter
        self.cafeteria = cafeteria
        self.customer_id = customer_id
        self.item = item
        self.quantity = quantity
        self.price = price*quantity*(1-(discount//100))
        #for simplicity, we assume all orders are accepted as the check is done before creating the order
        self.status = "Accepted"
        Order.class_counter+=1
        
    def __str__(self):
        return f"Order {self.order_id} by {self.customer_id} for {self.quantity} {self.item}(s) for {self.price}dkk"	
    
    def total_price(self):
        return self.price
    
    def complete(self):
        self.status = "Completed"
        print(f"Order {self.order_id} completed")
    
    def cancel(self):
        self.status = "Cancelled"
        #Refund
        for student in self.cafeteria.university.students:
            if student.customer_id == self.customer_id:
                student.balance+=self.price
        print(f"Order {self.order_id} cancelled")
        
    

#### University
